In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark Feedforward neural network example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/23 00:07:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/23 00:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/23 00:07:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load training data
df = spark.read.format('com.databricks.spark.csv').\
                               options(header='true', \
                               inferschema='true').load("./data/Wine_dataset.csv",header=True);
df.show(5)    

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|style|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      5|  red|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      5|  red|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      5|  red|
|         11.2|            0.28|       0.56|           1.9|    0

In [3]:
df.printSchema()

root
 |-- fixed_acidity: double (nullable = true)
 |-- volatile_acidity: double (nullable = true)
 |-- citric_acid: double (nullable = true)
 |-- residual_sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free_sulfur_dioxide: double (nullable = true)
 |-- total_sulfur_dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: integer (nullable = true)
 |-- style: string (nullable = true)



In [4]:
# Convert to float format
def string_to_float(x):
    return float(x)

# 
def condition(r):
    if (0<= r <= 4):
#        label = "low"
        label=0
    elif(4< r <= 6):
#        label = "medium"
        label=1
    else: 
#        label = "high"
        label=2
    return label

In [5]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType
string_to_float_udf = udf(string_to_float, DoubleType())
quality_udf = udf(lambda x: condition(x), StringType())

In [6]:
#df= df.withColumn("quality", string_to_float_udf("quality")).withColumn("Cquality", quality_udf("quality"))
df= df.withColumn("quality", quality_udf("quality"))

In [7]:
df.printSchema()

root
 |-- fixed_acidity: double (nullable = true)
 |-- volatile_acidity: double (nullable = true)
 |-- citric_acid: double (nullable = true)
 |-- residual_sugar: double (nullable = true)
 |-- chlorides: double (nullable = true)
 |-- free_sulfur_dioxide: double (nullable = true)
 |-- total_sulfur_dioxide: double (nullable = true)
 |-- density: double (nullable = true)
 |-- pH: double (nullable = true)
 |-- sulphates: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- quality: string (nullable = true)
 |-- style: string (nullable = true)



In [8]:
df.show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----+
|fixed_acidity|volatile_acidity|citric_acid|residual_sugar|chlorides|free_sulfur_dioxide|total_sulfur_dioxide|density|  pH|sulphates|alcohol|quality|style|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+-------+-----+
|          7.4|             0.7|        0.0|           1.9|    0.076|               11.0|                34.0| 0.9978|3.51|     0.56|    9.4|      1|  red|
|          7.8|            0.88|        0.0|           2.6|    0.098|               25.0|                67.0| 0.9968| 3.2|     0.68|    9.8|      1|  red|
|          7.8|            0.76|       0.04|           2.3|    0.092|               15.0|                54.0|  0.997|3.26|     0.65|    9.8|      1|  red|
|         11.2|            0.28|       0.56|           1.9|    0

In [9]:
# convert the data to dense vector
def transData(data):
    return data.rdd.map(lambda r: [r[-1], Vectors.dense(r[:-1])]).toDF(['label','features'])

In [10]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

data= transData(df)
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  red|[7.4,0.7,0.0,1.9,...|
|  red|[7.8,0.88,0.0,2.6...|
|  red|[7.8,0.76,0.04,2....|
|  red|[11.2,0.28,0.56,1...|
|  red|[7.4,0.7,0.0,1.9,...|
|  red|[7.4,0.66,0.0,1.8...|
|  red|[7.9,0.6,0.06,1.6...|
|  red|[7.3,0.65,0.0,1.2...|
|  red|[7.8,0.58,0.02,2....|
|  red|[7.5,0.5,0.36,6.1...|
|  red|[6.7,0.58,0.08,1....|
|  red|[7.5,0.5,0.36,6.1...|
|  red|[5.6,0.615,0.0,1....|
|  red|[7.8,0.61,0.29,1....|
|  red|[8.9,0.62,0.18,3....|
|  red|[8.9,0.62,0.19,3....|
|  red|[8.5,0.28,0.56,1....|
|  red|[8.1,0.56,0.28,1....|
|  red|[7.4,0.59,0.08,4....|
|  red|[7.9,0.32,0.51,1....|
+-----+--------------------+
only showing top 20 rows



In [11]:
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
labelIndexed=labelIndexer.transform(data)

In [21]:
labelIndexed.select("indexedLabel").distinct().show()

+------------+
|indexedLabel|
+------------+
|         0.0|
|         1.0|
+------------+



In [13]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
#featureIndexer =VectorIndexer(inputCol="features", \
#                              outputCol="indexedFeatures", \
#                              maxCategories=4).fit(data)

#featureIndexer.transform(labelIndexer).show(6)   

In [14]:
labelIndexed.printSchema()

root
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- indexedLabel: double (nullable = false)



In [15]:
# Split the data into train and test
(trainingData, testData) = labelIndexed.randomSplit([0.6, 0.4])

In [16]:
#data.show()

In [52]:
# specify layers for the neural network:
# input layer of size 12 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = [12, 8, 8, 4, 4 , 2]

# create the trainer and set its parameters
FNN = MultilayerPerceptronClassifier(labelCol="indexedLabel", featuresCol="features",\
                                         maxIter=100, layers=layers, blockSize=128, seed=1234)

In [53]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [54]:
# Chain indexers and forest in a Pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[FNN, labelConverter])

In [55]:
# train the model
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [56]:
# Make predictions.
predictions = model.transform(testData)

In [57]:
predictions.show()

+-----+--------------------+------------+--------------------+--------------------+----------+--------------+
|label|            features|indexedLabel|       rawPrediction|         probability|prediction|predictedLabel|
+-----+--------------------+------------+--------------------+--------------------+----------+--------------+
|  red|[4.9,0.42,0.0,2.1...|         1.0|[-0.2534875665578...|[0.59500158259152...|       0.0|         white|
|  red|[5.0,0.38,0.01,1....|         1.0|[1.00487245816899...|[0.95402340786992...|       0.0|         white|
|  red|[5.0,0.4,0.5,4.3,...|         1.0|[2.07781417132670...|[0.99459239015693...|       0.0|         white|
|  red|[5.0,0.42,0.24,2....|         1.0|[0.20351391080495...|[0.79431114987673...|       0.0|         white|
|  red|[5.0,0.74,0.0,1.2...|         1.0|[-1.8076020594141...|[0.05150551271183...|       1.0|           red|
|  red|[5.0,1.02,0.04,1....|         1.0|[-1.9245379442446...|[0.04066453217184...|       1.0|           red|
|  red|[5.

In [58]:
# Select example rows to display.
predictions.select("features","label","predictedLabel").show(5)

+--------------------+-----+--------------+
|            features|label|predictedLabel|
+--------------------+-----+--------------+
|[4.9,0.42,0.0,2.1...|  red|         white|
|[5.0,0.38,0.01,1....|  red|         white|
|[5.0,0.4,0.5,4.3,...|  red|         white|
|[5.0,0.42,0.24,2....|  red|         white|
|[5.0,0.74,0.0,1.2...|  red|           red|
+--------------------+-----+--------------+
only showing top 5 rows



In [59]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Predictions accuracy = %g, Test Error = %g" % (accuracy,(1.0 - accuracy)))

Predictions accuracy = 0.976046, Test Error = 0.0239544
